In [21]:
import geopandas as gpd
from shapely.geometry import Point
import pandas as pd
import numpy as np
import geopandas as gpd
from pathlib import Path

from PIL import Image
import matplotlib.pyplot as plt

data_path = Path('London_UK')

# Load data
panoids = gpd.read_file(data_path / 'panoids/panoids.geojson')
network = gpd.read_file(data_path / 'network/edges/edges.geojson')

In [22]:
# Create GeoDataFrame for Images
geometry = [Point(xy) for xy in zip(images_df['longitude'], images_df['latitude'])]
images_gdf = gpd.GeoDataFrame(images_df, geometry=geometry, crs=streets.crs)

In [25]:
# Perform Spatial Join
matched = gpd.sjoin_nearest(images_gdf, streets, how='left', distance_col='distance')

# Now 'matched' contains street information for each image
matched[['image_id', 'longitude', 'latitude', 'distance'][:5][0:2]]
matched_first_row = matched.iloc[[100]]

print(matched_first_row[['image_id', 'longitude', 'latitude', 'distance']])


C:\Users\fmaca\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\geopandas\array.py:403: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(


    image_id  longitude   latitude  distance
83      83.0  -0.109446  51.511078  0.000016


In [36]:
import geopandas as gpd
from shapely.geometry import Point
from pathlib import Path
import pandas as pd

def find_closest_street_photo(street_input, latitude=None, longitude=None):
    """
    Find the closest street photo ID for a given street or manual coordinates.
    
    Parameters:
    - street_input: Street name (optional)
    - latitude: Manual latitude (optional)
    - longitude: Manual longitude (optional)
    
    Returns:
    - Closest image details
    """
    # Predefined coordinates for some London streets
    street_coordinates = {
        'Newgate Street': (51.5178, -0.0975),
        'Oxford Street': (51.5146, -0.1424),
        'Piccadilly': (51.5097, -0.1338),
        'Downing Street': (51.5034, -0.1276),
        # Add more streets and their coordinates as needed
    }
    
    # Determine coordinates
    if latitude and longitude:
        # Use manually provided coordinates
        target_lat, target_lon = latitude, longitude
    elif street_input in street_coordinates:
        # Use predefined coordinates for known streets
        target_lat, target_lon = street_coordinates[street_input]
    else:
        raise ValueError(f"No coordinates found for {street_input}. Please provide manual latitude and longitude.")
    
    # Load the data
    data_path = Path('London_UK')
    
    # Load images data
    panoids = gpd.read_file(data_path / 'panoids/panoids.geojson')
    
    # Create target point
    target_point = Point(target_lon, target_lat)
    
    # Create GeoDataFrame for images
    geometry = [Point(xy) for xy in zip(panoids['lng'], panoids['lat'])]
    images_gdf = gpd.GeoDataFrame(panoids, geometry=geometry, crs="EPSG:4326")
    
    # Calculate distances
    images_gdf['distance'] = images_gdf.geometry.apply(lambda geom: geom.distance(target_point) * 111)  # Approximate kilometers
    
    # Find the closest image
    closest_image = images_gdf.loc[images_gdf['distance'].idxmin()]
    
    return {
        'panoid': closest_image['panoid'],
        'distance': closest_image['distance'],
        'latitude': closest_image['lat'],
        'longitude': closest_image['lng'],
        'year': closest_image['year'],
        'month': closest_image['month'],
        'images': {
            'side_a': closest_image['im_side_a'],
            'front': closest_image['im_front'],
            'side_b': closest_image['im_side_b'],
            'back': closest_image['im_back']
        },
        'url_side_a': closest_image['url_side_a'],
        'url_front': closest_image['url_front']
    }

# Example usage
try:
    # Method 1: Use a known street name
    result = find_closest_street_photo('Oxford Street')
    print("Method 1 (Street Name):")
    print(f"Closest Panoid: {result['panoid']}")
    print(f"Distance: {result['distance']:.2f} km")
    print(f"Image Location: ({result['latitude']}, {result['longitude']})")
    print(f"Image Filenames: {result['images']}")
    
    # Method 2: Use manual coordinates
    result = find_closest_street_photo(latitude=51.5178, longitude=-0.0975)
    print("\nMethod 2 (Manual Coordinates):")
    print(f"Closest Panoid: {result['panoid']}")
    print(f"Distance: {result['distance']:.2f} km")
    print(f"Image Location: ({result['latitude']}, {result['longitude']})")
    print(f"Image Filenames: {result['images']}")

except Exception as e:
    print(f"An error occurred: {e}")

Method 1 (Street Name):
Closest Panoid: pe-JMFDLDA9AdPWZfwrTZw
Distance: 3.24 km
Image Location: (51.51754651903001, -0.1133442997421481)
Image Filenames: {'side_a': 'image_2712_s_a.png', 'front': 'image_2712_f.png', 'side_b': 'image_2712_s_b.png', 'back': 'image_2712_b.png'}
An error occurred: find_closest_street_photo() missing 1 required positional argument: 'street_input'


In [34]:
import geopandas as gpd
from pathlib import Path
import pandas as pd

def diagnose_panoids_data():
    """
    Comprehensive diagnosis of panoids GeoJSON file
    """
    # Load the data
    data_path = Path('London_UK')
    
    try:
        # Load images data
        panoids = gpd.read_file(data_path / 'panoids/panoids.geojson')
        
        # Print comprehensive information
        print("=" * 50)
        print("PANOIDS DATAFRAME COMPREHENSIVE DIAGNOSTICS")
        print("=" * 50)
        
        # Full column names
        print("\n1. COLUMN NAMES:")
        print("-" * 20)
        for col in panoids.columns:
            print(col)
        
        # First few rows
        print("\n2. FIRST FEW ROWS:")
        print("-" * 20)
        print(panoids.head().to_string())
        
        # Data types
        print("\n3. COLUMN DATA TYPES:")
        print("-" * 20)
        print(panoids.dtypes)
        
        # Check for ID-like columns
        print("\n4. POTENTIAL ID COLUMNS:")
        print("-" * 20)
        potential_id_cols = [col for col in panoids.columns if 'id' in col.lower()]
        print(potential_id_cols)
        
    except Exception as e:
        print(f"An error occurred while diagnosing the data: {e}")

# Run the diagnostic function
diagnose_panoids_data()

PANOIDS DATAFRAME COMPREHENSIVE DIAGNOSTICS

1. COLUMN NAMES:
--------------------
panoid
year
month
owner
lat
lng
ask_lng
ask_lat
consulted
dist
angle
url_side_a
url_front
url_side_b
url_back
im_side_a
im_front
im_side_b
im_back
geometry

2. FIRST FEW ROWS:
--------------------
                   panoid  year  month   owner        lat       lng   ask_lng    ask_lat  consulted       dist   angle                                                                                                                                              url_side_a                                                                                                                                              url_front                                                                                                                                              url_side_b                                                                                                                                                url